Section 1: Using SQL and Python in Snowflake
1.	Add the Sample Movie Rental Database from Planet Data to your Snowflake account using the Snowflake Marketplace
2.	Write a query to return a dataset with film and rental, and payment information. Your dataset should have multiple rows per film, one for each time the film was rented and the amount spent on each rental. Create a dataframe with this information
3.	Create a dataframe from the customer table


In [ ]:
-- starting with a SQL query to build the dataset from which we'll conduct all our base analytics

USE DATABASE MOVIES_5740;
USE SCHEMA PUBLIC;

WITH payment_per_rental AS (
  SELECT
      rental_id,
      SUM(amount) AS amount_spent
  FROM PAYMENT
  GROUP BY rental_id
)
SELECT
    f.film_id,
    f.title,
    f.description,
    i.inventory_id,
    r.rental_id,
    r.rental_date,
    r.return_date,
    p.amount_spent,
    pay.customer_id
FROM FILM f
JOIN INVENTORY i
  ON i.film_id = f.film_id
JOIN RENTAL r
  ON r.inventory_id = i.inventory_id
LEFT JOIN payment_per_rental p
  ON p.rental_id = r.rental_id
LEFT JOIN PAYMENT pay
  ON pay.rental_id = r.rental_id
ORDER BY f.title, r.rental_date, r.rental_id;



In [ ]:
# Next, conduct the same query from within Python so we can move our dataset in a Pandas dataframe

from snowflake.snowpark.context import get_active_session

# Get the injected Snowpark session for this notebook
session = get_active_session()


session.sql("USE DATABASE MOVIES_5740").collect()
session.sql("USE SCHEMA PUBLIC").collect()

# Same SQL statement as above
film_rentals_sp = session.sql("""
WITH payment_per_rental AS (
    SELECT
        rental_id,
        SUM(amount) AS amount_spent
    FROM PAYMENT
    GROUP BY rental_id
)
SELECT
    f.film_id,
    f.title,
    f.description,
    r.rental_id,
    r.rental_date,
    r.return_date,
    p.amount_spent
FROM FILM f
JOIN INVENTORY i
    ON i.film_id = f.film_id
JOIN RENTAL r
    ON r.inventory_id = i.inventory_id
LEFT JOIN payment_per_rental p
    ON p.rental_id = r.rental_id
ORDER BY f.title, r.rental_date
""")

#create the Pandas dataframe from this query
film_rentals_df = film_rentals_sp.to_pandas()
print(film_rentals_df.head(10))

In [ ]:
# Create a customer dataframe

from snowflake.snowpark.context import get_active_session

# Get the active Snowpark session
session = get_active_session()

# Create a Snowpark DataFrame for the CUSTOMER table
customers_sp = session.table("CUSTOMER")

# Convert to Pandas
customers_df = customers_sp.to_pandas()

# Preview first 10 rows
print(customers_df.head(10))
print(f"Rows: {len(customers_df):,}")

Section 2: Manipulating dataframes
1.	Create a column for customer name that has the first name and last name in the same column. 
2.	Remove any inactive customers from the dataframe
Hint: use the active field
3.	Change the email addresses to be ‘joe.person@wustl.edu’, but only when their store_id is an even number
Hint: use apply to run a function over the dataframe, don’t forget to select the correct axis


In [ ]:
-- First, create the SQL query to see if we have it right, with the appropriate JOINs for building the desired table
WITH payment_per_rental AS (
    SELECT
        rental_id,
        SUM(amount) AS amount_spent
    FROM PAYMENT
    GROUP BY rental_id
)
SELECT
    f.film_id,
    f.title,
    f.description,
    f.mpaa_rating,
    f.rental_rate,
    i.inventory_id,
    r.rental_id,
    r.rental_date,
    r.return_date,
    p.amount_spent,
    c.customer_id,
    c.first_name,
    c.last_name,
    c.active,
    c.email,
    c.store_id
FROM FILM f
JOIN INVENTORY i
    ON i.film_id = f.film_id
JOIN RENTAL r
    ON r.inventory_id = i.inventory_id
JOIN PAYMENT pay
    ON pay.rental_id = r.rental_id
JOIN CUSTOMER c
    ON c.customer_id = pay.customer_id
LEFT JOIN payment_per_rental p
    ON p.rental_id = r.rental_id
ORDER BY f.title, r.rental_date, r.rental_id;

In [ ]:
# Next, in Python, use the query from above (that we tested) to create the dataset we desired, then put it into a Pandas dataframe.
# We'll also create the customer name column by putting the first name before the last name
# Then, we'll go through again and remove all the inactive customeers by checking if the ACTIVE column is inactive (value 0).  
#   Otherwise, we assume active.

from snowflake.snowpark.context import get_active_session

# Get active Snowpark session
session = get_active_session()

# Define the SQL string (triple quotes let us write multi-line SQL)
sql_query = """
WITH payment_per_rental AS (
    SELECT
        rental_id,
        SUM(amount) AS amount_spent
    FROM PAYMENT
    GROUP BY rental_id
)
SELECT
    f.film_id,
    f.title,
    f.description,
    f.mpaa_rating,
    f.rental_rate,
    i.inventory_id,
    r.rental_id,
    r.rental_date,
    r.return_date,
    p.amount_spent,
    c.customer_id,
    c.first_name,
    c.last_name,
    c.active,
    c.email,
    c.store_id
FROM FILM f
JOIN INVENTORY i
    ON i.film_id = f.film_id
JOIN RENTAL r
    ON r.inventory_id = i.inventory_id
JOIN PAYMENT pay
    ON pay.rental_id = r.rental_id
JOIN CUSTOMER c
    ON c.customer_id = pay.customer_id
LEFT JOIN payment_per_rental p
    ON p.rental_id = r.rental_id
ORDER BY f.title, r.rental_date, r.rental_id
"""

# Run the SQL and convert to Pandas
film_rentals_df = session.sql(sql_query).to_pandas()

# Create a combined customer_name column
film_rentals_df["CUSTOMER_NAME"] = (
    film_rentals_df["FIRST_NAME"] + " " + film_rentals_df["LAST_NAME"]
)

# Preview results
print(film_rentals_df[["TITLE", "CUSTOMER_ID", "FIRST_NAME", "LAST_NAME", "CUSTOMER_NAME"]].head(10))

In [ ]:
# Include only active customers (ACTIVE column is 1)
# Keep only rows where ACTIVE == 1
film_rentals_active_df = film_rentals_df[film_rentals_df["ACTIVE"] == 1]

# Preview
print(film_rentals_active_df.head(10))
print(f"Rows after filtering: {len(film_rentals_active_df):,}")


In [ ]:
# Change the email addresses to be ‘joe.person@wustl.edu’,
#   but only when their store_id is an even number 
#   Hint: use apply to run a function over the dataframe, don’t forget to select the correct axis

#define a function that replaces the e-mail address, but only if the store_id is an even number (modulus zero)
def replace_email(row):
    if row["STORE_ID"] % 2 == 0:         # even store_id
        return "joe.person@wustl.edu"
    else:
        return row["EMAIL"]              # keep the original

#apply the function across the entire dataframe, but do it for the rows (axis is 1)
film_rentals_df["EMAIL"] = film_rentals_df.apply(replace_email, axis=1)

#quick check to see if we did it right
print(film_rentals_df[["CUSTOMER_ID", "STORE_ID", "EMAIL"]].head(10))


In [ ]:
# also do the e-mail address change in the customer dataframe
def replace_email(row):
    if row["STORE_ID"] % 2 == 0:        # even store_id
        return "joe.person@wustl.edu"
    else:
        return row["EMAIL"]

customers_df["EMAIL"] = customers_df.apply(replace_email, axis=1)
print(customers_df[["CUSTOMER_ID", "STORE_ID", "EMAIL"]].head(10))


Section 3: Visualizations
1.	How much does each customer tend to spend in aggregate?
Clarification: You want to first create total spend by customer, then you want to visualize that distribution, each customer being an observation. A box and whisker plot would be a good visualization
2.	What does the distribution of film revenue look like?
Clarification: You want to first calculate the revenue by film, you can sum the rental rate for each instance that the film was rented using the dataframe you created in section 1 part 2. A histogram would be a good visualization


In [ ]:
# Group by customer and calculate total spend
customer_spend_df = (
    film_rentals_df.groupby("CUSTOMER_ID")["AMOUNT_SPENT"]
    .sum()
    .reset_index()
    .rename(columns={"AMOUNT_SPENT": "TOTAL_SPEND"})
)

print(customer_spend_df.head())

In [ ]:
import matplotlib.pyplot as plt

# Aggregate spend per customer
customer_spend_df = (
    film_rentals_df.groupby("CUSTOMER_ID")["AMOUNT_SPENT"]
    .sum()
    .reset_index()
    .rename(columns={"AMOUNT_SPENT": "TOTAL_SPEND"})
)

# Plot box and whisker
plt.figure(figsize=(8,6))
plt.boxplot(customer_spend_df["TOTAL_SPEND"], vert=True, patch_artist=True)

plt.title("Distribution of Total Spend per Customer")
plt.ylabel("Total Spend ($)")
plt.grid(axis="y", linestyle="--", alpha=0.7)

# This ensures the chart renders in Snowflake Notebooks / Jupyter
plt.show()


In [ ]:
# Aggregate total revenue by film
film_revenue_df = (
    film_rentals_df.groupby(["FILM_ID", "TITLE"])["AMOUNT_SPENT"]
    .sum()
    .reset_index()
    .rename(columns={"AMOUNT_SPENT": "TOTAL_REVENUE"})
)

print(film_revenue_df.head())

In [ ]:
import matplotlib.pyplot as plt

# generate a bar chart of the revenue/film by counting the number of films
plt.figure(figsize=(10,6))
plt.hist(film_revenue_df["TOTAL_REVENUE"], bins=30, edgecolor="black")

plt.title("Distribution of Total Film Revenue")
plt.xlabel("Total Revenue per Film ($)")
plt.ylabel("Number of Films")
plt.grid(axis="y", linestyle="--", alpha=0.7)

plt.show()

Section 4: Analysis
Clarification: You do not need to use a statistical test to answer the following questions. Please use a visualization and interpret what see. Note the averages, counts, or sums where applicable, and the interpretation
1.	On average, is the rental rate the same across movie ratings, treat each film as an observation?
2.	Across the various film ratings, are we observing the same number of movies rented at store 2 and store 1?
3.	On average, do films with a character of a ‘robot’ generate the same amount of revenue (use rental rate) as films that feature a ‘teacher’?
Hint: sum the rental rate by film across rentals to get revenue by film


In [ ]:
# Each film is one observation → keep film_id, title, mpaa_rating, rental_rate
film_rates_df = film_rentals_df.drop_duplicates(subset=["FILM_ID"])[
    ["FILM_ID", "TITLE", "MPAA_RATING", "RENTAL_RATE"]
]

print(film_rates_df.head())

In [ ]:
avg_rate_by_rating = (
    film_rates_df.groupby("MPAA_RATING")["RENTAL_RATE"]
    .mean()
    .reset_index()
    .rename(columns={"RENTAL_RATE": "AVG_RENTAL_RATE"})
)

print(avg_rate_by_rating)

In [ ]:
#try a simple bar chart
import matplotlib.pyplot as plt

plt.figure(figsize=(8,6))
plt.bar(avg_rate_by_rating["MPAA_RATING"], avg_rate_by_rating["AVG_RENTAL_RATE"], color="skyblue", edgecolor="black")

plt.title("Average Rental Rate by Film Rating")
plt.xlabel("Film Rating")
plt.ylabel("Average Rental Rate ($)")
plt.grid(axis="y", linestyle="--", alpha=0.7)

plt.show()

With my eyes, I do not see any significance to the impact of rating on rental rate.  In fact, it looks very evenly distributed.

However, I want to assure it's not a visual trick of some sort.  So, I'll run an Analysis of Variance 

In [ ]:
# One row per film (avoids duplicate rentals)
film_rates_df = film_rentals_df.drop_duplicates(subset=["FILM_ID"])[
    ["FILM_ID", "TITLE", "MPAA_RATING", "RENTAL_RATE"]
]

#continue as before
avg_rate_by_rating = (
    film_rates_df.groupby("MPAA_RATING")["RENTAL_RATE"]
    .mean()
    .reset_index()
    .rename(columns={"RENTAL_RATE": "AVG_RENTAL_RATE"})
)

print(avg_rate_by_rating)

In [ ]:
# One row per film (avoids duplicate rentals)
film_rates_df = film_rentals_df.drop_duplicates(subset=["FILM_ID"])[
    ["FILM_ID", "TITLE", "MPAA_RATING", "RENTAL_RATE"]
]

# run ANOVA with scipy
from scipy.stats import f_oneway

# Build a list of rental_rate values, one list per rating
groups = [
    film_rates_df.loc[film_rates_df["MPAA_RATING"] == rating, "RENTAL_RATE"].values
    for rating in film_rates_df["MPAA_RATING"].unique()
]

# Perform one-way ANOVA
f_stat, p_val = f_oneway(*groups)

print("ANOVA F-statistic:", f_stat)
print("p-value:", p_val)

p-value < 0.05 → statistically significant difference in average rental rates between at least one pair of ratings.
p-value ≥ 0.05 → no evidence that rental rates differ by rating (any variation is likely random).

Our p-value is greater than 0.05 (much greater, in fact).  So, this confirms my visual observation from the bar chart above.

Section 4: Analysis
Clarification: You do not need to use a statistical test to answer the following questions. Please use a visualization and interpret what see. Note the averages, counts, or sums where applicable, and the interpretation

2.	Across the various film ratings, are we observing the same number of movies rented at store 2 and store 1?
    


In [ ]:
# Count number of rentals by rating and store
rating_store_counts = (
    film_rentals_df.groupby(["MPAA_RATING", "STORE_ID"])["RENTAL_ID"]
    .count()
    .reset_index()
    .rename(columns={"RENTAL_ID": "RENTAL_COUNT"})
)

print(rating_store_counts.head())

In [ ]:
import matplotlib.pyplot as plt

pivot_counts = rating_store_counts.pivot(index="MPAA_RATING", columns="STORE_ID", values="RENTAL_COUNT")

pivot_counts.plot(kind="bar", figsize=(10,6))

plt.title("Number of Rentals by Film Rating and Store")
plt.xlabel("Film Rating")
plt.ylabel("Number of Rentals")
plt.legend(title="Store")
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.show()

In [ ]:
from scipy.stats import chi2_contingency

# Build contingency table: rows = ratings, cols = stores
contingency_table = pivot_counts.fillna(0).astype(int)

chi2, p, dof, expected = chi2_contingency(contingency_table)

print("Chi-square statistic:", chi2)
print("p-value:", p)

again, p-value is greater than 0.05, so we do not have a significant difference - which confirms our visual check (they look similar)

3.	On average, do films with a character of a ‘robot’ generate the same amount of revenue (use rental rate) as films that feature a ‘teacher’?
Hint: sum the rental rate by film across rentals to get revenue by film


In [ ]:
# Step 1: revenue per film
film_revenue_df = (
    film_rentals_df.groupby(["FILM_ID", "TITLE", "DESCRIPTION"])["RENTAL_RATE"]
    .sum()
    .reset_index()
    .rename(columns={"RENTAL_RATE": "TOTAL_REVENUE"})
)

print(film_revenue_df.head())

In [ ]:
# Make sure DESCRIPTION is a string and lowercase for consistent matching
film_revenue_df["DESCRIPTION"] = film_revenue_df["DESCRIPTION"].astype(str).str.lower()

# Create boolean flags
film_revenue_df["HAS_ROBOT"] = film_revenue_df["DESCRIPTION"].str.contains("robot", na=False)
film_revenue_df["HAS_TEACHER"] = film_revenue_df["DESCRIPTION"].str.contains("teacher", na=False)

# Quick check of how many films match each
print("Robot films:", film_revenue_df["HAS_ROBOT"].sum())
print("Teacher films:", film_revenue_df["HAS_TEACHER"].sum())


robot_revenue = film_revenue_df.loc[film_revenue_df["HAS_ROBOT"], "TOTAL_REVENUE"]
teacher_revenue = film_revenue_df.loc[film_revenue_df["HAS_TEACHER"], "TOTAL_REVENUE"]

print("Number of robot films:", len(robot_revenue))
print("Number of teacher films:", len(teacher_revenue))

print("Average robot film revenue: ${:,.2f}".format(robot_revenue.mean()))
print("Average teacher film revenue: ${:,.2f}".format(teacher_revenue.mean()))

In [ ]:
from scipy.stats import ttest_ind

t_stat, p_val = ttest_ind(robot_revenue, teacher_revenue, equal_var=False)  # Welch’s t-test
print("t-statistic:", t_stat)
print("p-value:", p_val)

There is no significant difference give the p-value is greater than 0.05

Section 5: Analysis
Its late 2005, and your boss at the DVD rental company wants to know how effective his customer promotion program was. He tells you, ‘I want you to give me some descriptive information about how much the customers spent before and after the program started. Were the spending habits similar? Did they differ? Did the program help or make things worse?’
1.	What is the outcome?
2.	What is the main effect/predictor he wants to understand the impact of?
3.	What is the hypothesis?
Lucky for you, your boss already asked Ted in Bethesda to give you a query for how to get the information. 
Query:
        with b4 as (
            select p.customer_id, sum(p.amount) as Payment_before
            from rental r 
            left outer join payment p on p.rental_id = r.rental_id
            where rental_date < cast('2005-07-01' as timestamp) and
                amount is not null
            group by p.customer_id),
        aft as (
            select p.customer_id, sum(p.amount) as Payment_after
            from rental r 
            left outer join payment p on p.rental_id = r.rental_id
            where rental_date >= cast('2005-07-01' as timestamp) and
                amount is not null
            group by p.customer_id
        )
        select distinct c.customer_id, store_id, first_name, last_name,
            active, payment_before, payment_after
        from customer c
        left outer join b4 r on r.customer_id = c.customer_id
        left outer join aft a on a.customer_id = c.customer_id
        where payment_after is not null and payment_before is not null
Plus, the statistician you work with has some suggestions for how to give your boss what he wants. Query the data from your container and put it in a Pandas dataframe. Then follow the statistician’s suggestions.
4.	Compute summary statistics and create histograms of the payment_before and payment_after variables. (Try using describe()in pandas).
5.	Compute the correlation between these two variables and create a scatterplot
6.	Compute a variable which is the difference between the amounts spent before and after the program started:  payment_after – payment_before. 
7.	Generate a histogram of the difference and conduct a one-sample t-test.
8.	Interpret your results


In [ ]:
-- Query from the problem statement.
with b4 as ( select p.customer_id, sum(p.amount) as Payment_before from rental r left outer join payment p on p.rental_id = r.rental_id where rental_date < cast('2005-07-01' as timestamp) and amount is not null group by p.customer_id), aft as ( select p.customer_id, sum(p.amount) as Payment_after from rental r left outer join payment p on p.rental_id = r.rental_id where rental_date >= cast('2005-07-01' as timestamp) and amount is not null group by p.customer_id ) select distinct c.customer_id, store_id, first_name, last_name, active, payment_before, payment_after from customer c left outer join b4 r on r.customer_id = c.customer_id left outer join aft a on a.customer_id = c.customer_id where payment_after is not null and payment_before is not null

In [ ]:
# now, repeat the query, but in Python so we can create a Pandas __DataFrame__

from snowflake.snowpark.context import get_active_session

# Get the active Snowpark session
session = get_active_session()

# Define your SQL query
sql_query = """
WITH b4 AS (
    SELECT 
        p.customer_id, 
        SUM(p.amount) AS payment_before
    FROM rental r 
    LEFT OUTER JOIN payment p 
        ON p.rental_id = r.rental_id
    WHERE rental_date < CAST('2005-07-01' AS TIMESTAMP)
      AND amount IS NOT NULL
    GROUP BY p.customer_id
), 
aft AS (
    SELECT 
        p.customer_id, 
        SUM(p.amount) AS payment_after
    FROM rental r 
    LEFT OUTER JOIN payment p 
        ON p.rental_id = r.rental_id
    WHERE rental_date >= CAST('2005-07-01' AS TIMESTAMP)
      AND amount IS NOT NULL
    GROUP BY p.customer_id
)
SELECT DISTINCT 
    c.customer_id, 
    c.store_id, 
    c.first_name, 
    c.last_name, 
    c.active, 
    r.payment_before, 
    a.payment_after
FROM customer c
LEFT OUTER JOIN b4 r 
    ON r.customer_id = c.customer_id
LEFT OUTER JOIN aft a 
    ON a.customer_id = c.customer_id
WHERE payment_after IS NOT NULL 
  AND payment_before IS NOT NULL
"""

# Run the query and convert to Pandas
payments_df = session.sql(sql_query).to_pandas()

# Preview results
print(payments_df.head())
print(f"Rows: {len(payments_df):,}")

In [ ]:
# Summary statistics for before/after payments
print("Summary statistics:\n")
print(payments_df[["PAYMENT_BEFORE", "PAYMENT_AFTER"]].describe())

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12,5))

# Histogram: payment_before
plt.subplot(1,2,1)
plt.hist(payments_df["PAYMENT_BEFORE"].dropna(), bins=30, edgecolor="black")
plt.title("Distribution of Payments Before July 2005")
plt.xlabel("Total Payment Before")
plt.ylabel("Number of Customers")

# Histogram: payment_after
plt.subplot(1,2,2)
plt.hist(payments_df["PAYMENT_AFTER"].dropna(), bins=30, edgecolor="black")
plt.title("Distribution of Payments After July 2005")
plt.xlabel("Total Payment After")
plt.ylabel("Number of Customers")

plt.tight_layout()
plt.show()

In [ ]:
# Compute the correlation between these two variables and create a scatterplot
# Correlation matrix
corr = payments_df[["PAYMENT_BEFORE", "PAYMENT_AFTER"]].corr()

print("Correlation between payment_before and payment_after:")
print(corr)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(7,6))
plt.scatter(
    payments_df["PAYMENT_BEFORE"],
    payments_df["PAYMENT_AFTER"],
    alpha=0.6, edgecolor="black"
)

plt.title("Scatterplot of Payments Before vs After July 2005")
plt.xlabel("Payment Before (total per customer)")
plt.ylabel("Payment After (total per customer)")
plt.grid(True, linestyle="--", alpha=0.7)

plt.show()

In [ ]:
# Compute a variable which is the difference between the amounts spent before and after the program started: payment_after – payment_before

# Compute the difference
payments_df["PAYMENT_DIFF"] = payments_df["PAYMENT_AFTER"] - payments_df["PAYMENT_BEFORE"]

# Quick preview
print(payments_df[["CUSTOMER_ID", "PAYMENT_BEFORE", "PAYMENT_AFTER", "PAYMENT_DIFF"]].head())

In [ ]:
# Generate a histogram of the difference and conduct a one-sample t-test

import matplotlib.pyplot as plt

plt.figure(figsize=(8,6))
plt.hist(payments_df["PAYMENT_DIFF"].dropna(), bins=30, edgecolor="black")
plt.title("Distribution of Payment Difference (After – Before)")
plt.xlabel("Difference in Payments")
plt.ylabel("Number of Customers")
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.show()

In [ ]:
from scipy.stats import ttest_1samp

diff_values = payments_df["PAYMENT_DIFF"].dropna()

t_stat, p_val = ttest_1samp(diff_values, popmean=0)

print("One-sample t-test for mean difference vs 0")
print("t-statistic:", t_stat)
print("p-value:", p_val)
print("Average difference:", diff_values.mean())

Interpret The Results:

I arbitrarily drew the line at halfway through the year and assumed any impact from the program would've occured after July 1 of that year.  On average, the customers spent about $71.42 more after the program compared to before the program.  The t-test value of ~65 shows the difference is highly statistically significant.  This is a positive indication of the impact of the program and that the increased spending is very real, and not due to random chance.  Our conclusion is that the program had a strong positive impact on customer payments.

For a visual on this, see the above scatterplot which shows the movement "up" on the left side of the chart - indicating a greater spend after the program, than before. Furthermore, the distribution bar chart confirms a higher concentration of customer payment size after the program.